In [32]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import ast
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
MACEannots = []
annotStrList = []
stem = "/shared/3/projects/benlitterer/podcastData/annotation/label1000/MACE/outputs"
iter = 0 
for fPath in os.listdir(stem): 
    if "prediction" in fPath: 
        currFile = open(stem + "/" + fPath)
        cMACEannots = [int(item.strip("\n")) for item in currFile.readlines()]
        cAnnotStrList = [fPath.split(".")[0] for i in range(len(cMACEannots))] 
        MACEannots += cMACEannots
        annotStrList += cAnnotStrList 
        iter += 1
print(iter)

25


In [3]:
annotDf = pd.DataFrame({"MACEannots":MACEannots, "annotStr":annotStrList})
annotDf.shape

(1000, 2)

In [4]:
annotDf["MACEannots"].value_counts() 

MACEannots
3    427
1    328
2    245
Name: count, dtype: int64

In [5]:
#now get the data associated with their ids
inputDf = pd.read_json("/shared/3/projects/benlitterer/podcastData/annotation/label1000/MACE/inputsDf.jsonl", orient="records", lines=True)

In [6]:
Counter([2, 1, 3]).most_common(1)

[(2, 1)]

In [11]:
def getMostCommon(inList): 
    mostCommon, modalFreq = Counter(inList).most_common(1)[0]
    if modalFreq == 1: 
        mostCommon = 3
    return mostCommon 
     

mergedMatchesList = []
dfList = []

for annotStr in inputDf["annotStr"].unique(): 
    currDf = inputDf[inputDf["annotStr"] == annotStr]

    #the MACE annotation prediction 
    currDf["MACEannots"] = annotDf.loc[annotDf["annotStr"] == annotStr, "MACEannots"].to_list()

    #the modal annotation from the three raters 
    currDf["modalAnnots"] = currDf["annotation"].apply(getMostCommon)
    
    dfList.append(currDf)
    

/tmp/ipykernel_178344/4181722309.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currDf["MACEannots"] = annotDf.loc[annotDf["annotStr"] == annotStr, "MACEannots"].to_list()
/tmp/ipykernel_178344/4181722309.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currDf["modalAnnots"] = currDf["annotation"].apply(getMostCommon)
/tmp/ipykernel_178344/4181722309.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [12]:
annotDf = pd.concat(dfList, axis=0)

In [13]:
print(confusion_matrix(annotDf["MACEannots"], annotDf["modalAnnots"])) 
print(np.corrcoef(annotDf["MACEannots"], annotDf["modalAnnots"]))

[[323   0   5]
 [  1 235   9]
 [  1   1 425]]
[[1.         0.97676518]
 [0.97676518 1.        ]]


In [14]:
#find the rows from our original dataframe that correspond to the rows we trained with 
#11 minute load time... 
metadataDf = pd.read_json("/shared/3/projects/benlitterer/podcastData/processed/floydMonth/floydMonthDataClean.jsonl", orient="records", lines=True)

In [15]:
merged = pd.merge(annotDf.rename(columns={"id":"enclosure"}), metadataDf, on="enclosure", how="left") 

In [16]:
#now we can add on the specific entity that was used from the podcast rows that we have  
taskDf = pd.read_json("/shared/3/projects/benlitterer/podcastData/annotation/label1000/1075podsClassification.jsonl", orient="records", lines=True)

In [17]:
taskDf.columns

Index(['potentialOutPath', 'descClean', 'epDescClean', 'transcript',
       'transEnts', 'transTypes', 'rssUrl', 'id', 'cleanDatesLoc', 'epTitle',
       'title', 'transChunk', 'text', 'cleanEnt'],
      dtype='object')

In [18]:
merged = pd.merge(merged, taskDf[["id", "transEnts", "cleanEnt"]].rename(columns={"id":"enclosure", "transEnts":"annotEnt"}), on="enclosure", how="left")

In [19]:
mergedLong = merged.explode(["transEnts", "transStarts", "transEnds", "transTypes"])

In [20]:
mergedLong[["transEnts", "transStarts", "transEnds", "transTypes", "annotEnt", "cleanEnt"]]

,transEnts,transStarts,transEnds,transTypes,annotEnt,cleanEnt
0,the Read Play Talk,18,36,WORK_OF_ART,Jack Love,jack love
0,Lindsay Parris,69,83,PERSON,Jack Love,jack love
0,Jack Love,110,119,PERSON,Jack Love,jack love
0,every day,269,278,DATE,Jack Love,jack love
0,ISD,326,329,GPE,Jack Love,jack love
...,...,...,...,...,...,...
999,YouTube,35017,35024,ORG,Brooke Lopez,brooke lopez
999,every day,35062,35071,DATE,Brooke Lopez,brooke lopez
999,Amazon,35142,35148,ORG,Brooke Lopez,brooke lopez
999,Amazon,35169,35175,ORG,Brooke Lopez,brooke lopez


In [21]:
mergedMatches = mergedLong[mergedLong["annotEnt"] == mergedLong["transEnts"]]

In [22]:
mergedMatches.columns

Index(['enclosure', 'annotatorId', 'annotation', 'annotStr', 'MACEannots',
       'modalAnnots', 'potentialOutPath', 'transcript', 'wCount', 'rssUrl',
       'epTitle', 'epDescription', 'duration', 'pubDate', 'copyright',
       'itunes:type', 'itunes:complete', 'guid', 'itunes:explicit',
       'itunes:image', 'transDict', 'id', 'title', 'lastUpdate', 'link',
       'lastHttpStatus', 'dead', 'contentType', 'itunesId', 'originalUrl',
       'itunesAuthor', 'itunesOwnerName', 'explicit', 'imageUrl', 'itunesType',
       'generator', 'newestItemPubdate', 'language', 'oldestItemPubdate',
       'episodeCount', 'popularityScore', 'priority', 'createdOn',
       'updateFrequency', 'chash', 'host', 'newestEnclosureUrl', 'podcastGuid',
       'podDescription', 'category1', 'category2', 'category3', 'category4',
       'category5', 'category6', 'category7', 'category8', 'category9',
       'category10', 'newestEnclosureDuration', 'oldestItemDatetime',
       'cleanDates', 'cleanDatesLoc', 'par

In [23]:
keepCols = ['enclosure', 'annotatorId', 'annotation', 'annotStr', 'MACEannots',
       'modalAnnots', 'potentialOutPath', 'transcript', 'wCount', 'rssUrl',
       'epTitle', 'epDescription', 'duration', 'pubDate', 'copyright',
       'itunes:type', 'itunes:complete', 'guid', 'itunes:explicit',
       'itunes:image', 'transDict', 'id', 'title', 'itunesAuthor', 'language', 'episodeCount', 'podcastGuid',
       'podDescription', 'category1', 'category2', 'category3', 'category4',
       'category5', 'category6', 'category7', 'category8', 'category9',
       'category10', 'cleanDates', 'cleanDatesLoc', 'parsedDuration', 'hours', 'transEnts',
       'transStarts', 'transEnds', 'transTypes', 'descEnts', 'descStarts',
       'descEnds', 'descTypes', 'epDescEnts', 'epDescStarts', 'epDescEnds',
       'epDescTypes', 'totalGramCount', 'topGramCount', 'topGram', 'gramFrac',
       'annotEnt', 'cleanEnt']

In [24]:
mergedMatches = mergedMatches[keepCols]

In [25]:

mergedMatches["groundTruth"] = mergedMatches["MACEannots"].astype(int) - 1

#now extract snippets surrounding the text 
def getEnt(inRow): 
    return inRow["transcript"][inRow["transStarts"]:inRow["transEnds"]]

#this is just a sanity check 
mergedMatches["foundEnt"] = mergedMatches.apply(getEnt, axis=1)

#PERFECT! we find the entity in every case 
mergedMatches[mergedMatches["foundEnt"] != mergedMatches["annotEnt"]].shape

(0, 62)

In [26]:
BEFORE_BUFF = 200
AFTER_BUFF=200
#PUNCH IN HERE
def getSnippet(row): 
    #find where the entity starts quick 
   # row["snippetStart"] = trainDf.apply(lambda x: x["entSnippets"].lower().find(x["ent"].lower()), axis=1)
    
    snippet = row["transcript"]
    entStart = row["transStarts"]
    entEnd = row["transEnds"]


    beforeWords = snippet[0:entStart].split(" ")
    if len(beforeWords) >= BEFORE_BUFF: 
        buffStart = " ".join(beforeWords[-BEFORE_BUFF:]) 
    else: 
        buffStart = " ".join(beforeWords) 

    afterWords = snippet[entEnd:len(snippet)].split(" ")
    if len(afterWords) >= AFTER_BUFF: 
        buffEnd = " ".join(afterWords[:AFTER_BUFF]) 
    else: 
        buffEnd = " ".join(afterWords) 
    return [buffStart, snippet[entStart:entEnd], buffEnd]
            


In [27]:
mergedMatches[["left", "ent", "right"]] = pd.DataFrame(mergedMatches.apply(getSnippet, axis=1).tolist(), index=mergedMatches.index)

In [28]:
mergedMatches.head() 

,enclosure,annotatorId,annotation,annotStr,MACEannots,modalAnnots,potentialOutPath,transcript,wCount,rssUrl,...,topGramCount,topGram,gramFrac,annotEnt,cleanEnt,groundTruth,foundEnt,left,ent,right
0,http://awscdn.podcasts.com/Episode-5-Interview...,"[57b8e70f35624400013d690c, 5fee5c55878eaa15ebc...","[1, 1, 1]","57b8e70f35624400013d690c,5fee5c55878eaa15ebc56...",1,1,/awscdn.podcasts.com/ei/httpawscdn.podcasts.co...,"Hello, welcome to the Read Play Talk podcast. ...",3392,https://www.podcasts.com/rss_feed/ffe19768b9ff...,...,9,to be a part,0.002657,Jack Love,jack love,0,Jack Love,"Hello, welcome to the Read Play Talk podcast. ...",Jack Love,. This podcast is a resource for all things Re...
1,https://anchor.fm/s/11a3f14/podcast/play/13893...,"[57b8e70f35624400013d690c, 5fee5c55878eaa15ebc...","[3, 3, 3]","57b8e70f35624400013d690c,5fee5c55878eaa15ebc56...",3,3,/anchor.fm/12/httpsanchor.fms11a3f14podcastpla...,-Seven days to play! -Welcome to Seven Days to...,5170,https://anchor.fm/s/11a3f14/podcast/rss,...,14,"dig, dig, dig, dig,",0.002711,Hugh Colon,hugh colon,2,Hugh Colon,-Seven days to play! -Welcome to Seven Days to...,Hugh Colon,", a color adventure. Hugh Colon, a grand story..."
1,https://anchor.fm/s/11a3f14/podcast/play/13893...,"[57b8e70f35624400013d690c, 5fee5c55878eaa15ebc...","[3, 3, 3]","57b8e70f35624400013d690c,5fee5c55878eaa15ebc56...",3,3,/anchor.fm/12/httpsanchor.fms11a3f14podcastpla...,-Seven days to play! -Welcome to Seven Days to...,5170,https://anchor.fm/s/11a3f14/podcast/rss,...,14,"dig, dig, dig, dig,",0.002711,Hugh Colon,hugh colon,2,Hugh Colon,-Seven days to play! -Welcome to Seven Days to...,Hugh Colon,", a grand story. Yep, would have been fun. Hug..."
2,https://anchor.fm/s/11cf1e60/podcast/play/1425...,"[57b8e70f35624400013d690c, 5fee5c55878eaa15ebc...","[1, 1, 1]","57b8e70f35624400013d690c,5fee5c55878eaa15ebc56...",1,1,/anchor.fm/21/httpsanchor.fms11cf1e60podcastpl...,What's up with Jess Sub? It's the only way to ...,385,https://anchor.fm/s/11cf1e60/podcast/rss,...,6,"rock, set, rock, set,",0.015789,Jess Sub,jess sub,0,Jess Sub,What's up with,Jess Sub,? It's the only way to wake up. We state our v...
2,https://anchor.fm/s/11cf1e60/podcast/play/1425...,"[57b8e70f35624400013d690c, 5fee5c55878eaa15ebc...","[1, 1, 1]","57b8e70f35624400013d690c,5fee5c55878eaa15ebc56...",1,1,/anchor.fm/21/httpsanchor.fms11cf1e60podcastpl...,What's up with Jess Sub? It's the only way to ...,385,https://anchor.fm/s/11cf1e60/podcast/rss,...,6,"rock, set, rock, set,",0.015789,Jess Sub,jess sub,0,Jess Sub,What's up with Jess Sub? It's the only way to ...,Jess Sub,"? Thanks for watching. And they foil, need no ..."


In [37]:
#we need to split on the basis of entitys 
#so get a list of entities 
uniqueEnts = mergedMatches["annotEnt"].unique()

In [51]:
#now split this randomly 
trainEnts, testEnts = train_test_split(uniqueEnts, train_size=.8, test_size=.2, random_state=42)

In [52]:
#TODO: now punch in and split off a test chunk 
trainData = mergedMatches[mergedMatches["annotEnt"].apply(lambda x: x in trainEnts)] 
testData = mergedMatches[mergedMatches["annotEnt"].apply(lambda x: x in testEnts)] 

In [53]:
print(trainData["MACEannots"].value_counts())
print(testData["MACEannots"].value_counts())

MACEannots
3    1128
2     451
1     427
Name: count, dtype: int64
MACEannots
3    254
1    105
2     84
Name: count, dtype: int64


In [54]:
print(trainData.shape)
print(testData.shape)

(2006, 65)
(443, 65)


In [56]:
#write these ground truth values
trainData.to_json("/shared/3/projects/benlitterer/podcastData/annotation/label1000/MACE/1000annotTrain.jsonl", orient="records", lines=True)
testData.to_json("/shared/3/projects/benlitterer/podcastData/annotation/label1000/MACE/1000annotTest.jsonl", orient="records", lines=True)